In [9]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import cv2 as cv
import warnings
from collections import OrderedDict
import dlib
import imutils
import matplotlib

In [10]:
warnings.filterwarnings(action='ignore')

In [11]:
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/drive/MyDrive/shape_predictor_68_face_landmarks.dat" "/content/shape_predictor_68_face_landmarks.dat"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
def cheek(img) :
    CHEEK_IDXS = OrderedDict([("left_cheek", (1, 2, 3, 4, 5, 48, 31)),
                            ("right_cheek", (11, 12, 13, 14, 15, 35, 54))
                            ])

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    img = imutils.resize(img, width=600)

    overlay = img.copy()
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    bounding_rects = []
    detections = detector(gray, 0)
    for k, d in enumerate(detections):
        shape = predictor(gray, d)
        for (_, name) in enumerate(CHEEK_IDXS.keys()):
            pts = np.zeros((len(CHEEK_IDXS[name]), 2), np.int32)
            for i, j in enumerate(CHEEK_IDXS[name]):
                pts[i] = [shape.part(j).x, shape.part(j).y]
            pts = pts.reshape((-1, 1, 2))
            cv.drawContours(overlay, [pts], -1, (255, 255, 255), -1, cv.LINE_AA)
            bounding_rects.append(cv.boundingRect(pts))
    height, width=overlay.shape[:2]

    for y in range(0, height):
        for x in range(0, width):
            b=overlay.item(y, x, 0)
            g=overlay.item(y, x, 1)
            r=overlay.item(y, x, 2)
            if b==255 and g==255 and r==255:
                continue
            overlay.itemset(y, x, 0, 0)
            overlay.itemset(y, x, 1, 0)
            overlay.itemset(y, x, 2, 0)
    output = 0
    output=cv.bitwise_and(img, overlay, output)
    black=0
    for y in range(0, height):
        for x in range(0, width):
            b=output.item(y, x, 0)
            g=output.item(y, x, 1)
            r=output.item(y, x, 2)
            if b==0 and g==0 and r==0:
                black+=1
            if b==255 and g==255 and r==255:
                output.itemset(y, x, 0, 0)
                output.itemset(y, x, 1, 0)
                output.itemset(y, x, 2, 0)
    if black > height*width-10 : return img
    return output


In [13]:
class Color :
    person_HSV = []

    def color_classifier(self, person_HSV) :
        self.H = float(person_HSV[0])
        self.S = float(person_HSV[1])
        self.V = float(person_HSV[2])
        diff = round(self.V - self.S, 2)
        print(self.H, diff)
        if self.H >= 20 and self.H <= 210 :
            if diff >= 68.25 :
                    self.ans = 0
                    # Spring
            elif diff < 68.25:
                    self.ans = 1
                    # Autumn
        elif (self.H >= 0 and self.H < 20) or (self.H > 210 and self.H < 360) :
            if diff >= 68.75 :
                    self.ans = 2
                    # Summer
            elif diff < 68.75:
                    self.ans = 3
                    # Winter

        else :
            self.ans = -1
            # 에러

        return self.ans


In [14]:
def color_ratio(clt) :
    numLabels = np.arange(0, len(np.unique(clt.labels_))+1)
    (hist, _) = np.histogram(clt.labels_, bins=numLabels)
    hist = hist.astype("float")
    hist /= hist.sum()
    return hist

In [15]:
def plot_colors(hist, centroids):
    bar = np.zeros((50, 300, 3), dtype = "uint8")
    startX = 0

    for (percent, color) in zip(hist, centroids):
        endX = startX + (percent * 300)
        cv.rectangle(bar, (int(startX), 0), (int(endX), 50),
            color.astype("uint8").tolist(), -1)
        startX = endX
    return bar

In [16]:
def skin_detector(img, file_name) :
    lower = np.array([0, 48, 80], dtype = "uint8")
    upper = np.array([20, 255, 255], dtype = "uint8")

    converted = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    skinMask = cv.inRange(converted, lower, upper)

    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (11,11))
    skinMask = cv.erode(skinMask, kernel, iterations = 2)
    skinMask = cv.dilate(skinMask, kernel, iterations = 2)

    skinMask = cv.GaussianBlur(skinMask, (3,3), 0)
    skin = cv.bitwise_and(img, img, mask = skinMask)

    result = skin
    img = cv.cvtColor(result, cv.COLOR_BGR2HLS)
    skin_img = img
    temp_img = cv.cvtColor(img, cv.COLOR_HLS2RGB)

    h, w, c = img.shape

    for i in range(h) :
        for j in range(w) :
            H = img[i][j][0]
            L = img[i][j][1]
            S = img[i][j][2]

            R = temp_img[i][j][0]
            G = temp_img[i][j][1]
            B = temp_img[i][j][2]

            if R==0 and G==0 and B==0:
                continue

            LS_ratio = L/S
            skin_pixel = bool((S>=50) and (LS_ratio > 0.5) and (LS_ratio < 3.0) and ((H <= 25) or (H >= 165)))
            temp_pixel = bool((R == G) and (G == B) and (R >= 220))

            if skin_pixel :
                if temp_pixel :
                    skin_img[i][j][0] = 0
                    skin_img[i][j][1] = 0
                    skin_img[i][j][2] = 0
                else :
                    pass
            else :
                skin_img[i][j][0] = 0
                skin_img[i][j][1] = 0
                skin_img[i][j][2] = 0

    skin_img = cv.cvtColor(skin_img, cv.COLOR_HLS2BGR)
    for i in range(h) :
        for j in range(w) :
            B = skin_img[i][j][0]
            G = skin_img[i][j][1]
            R = skin_img[i][j][2]

            bg_pixel = bool(B==0 and G==0 and R==0)

            if bg_pixel :
                skin_img[i][j][0] = 255
                skin_img[i][j][1] = 255
                skin_img[i][j][2] = 255
            else :
                pass

    cvt_img = cv.cvtColor(skin_img, cv.COLOR_BGR2RGB)
    cvt_img = cvt_img.reshape((cvt_img.shape[0]*cvt_img.shape[1], 3))
    k = 20
    clt = KMeans(n_clusters=k)
    clt.fit(cvt_img)


    hist = color_ratio(clt)
    temp = np.array(clt.cluster_centers_)

    del_index = hist.argmax()
    hist = np.delete(hist, del_index)
    temp = np.delete(temp, del_index, 0)


    try :
        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)

        del_index = np.argmin(hist)
        hist = np.delete(hist, del_index)
        temp = np.delete(temp, del_index, 0)
    except ValueError :
        print(file_name, "에러")
        pass

    hist = hist / hist.sum()
    bar = plot_colors(hist, temp)

    bar = cv.cvtColor(bar, cv.COLOR_BGR2RGB)

    return bar

In [17]:
def color_convert2(cheek):
    img=cheek
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

    R = []
    G = []
    B = []
    for i in img :
        for j in i :
            R.append(j[0])
            G.append(j[1])
            B.append(j[2])

    R_sum = 0
    G_sum = 0
    B_sum = 0

    # 각 R, G, B의 합계 구하기
    for i in range(len(R)) :
        R_sum += R[i]
        G_sum += G[i]
        B_sum += B[i]

    R_avg = int(round((R_sum / len(R)), 0))  # R값 평균
    G_avg = int(round((G_sum / len(G)), 0))  # G값 평균
    B_avg = int(round((B_sum / len(B)), 0))  # B값 평균

    img_avg = img

    for i in img_avg :
        for j in i :
            j[0] = R_avg
            j[1] = G_avg
            j[2] = B_avg

    bgr_img_avg = cv.cvtColor(img_avg, cv.COLOR_RGB2BGR)
    bgr_img_avg=cv.resize(bgr_img_avg, (50, 50))
    return bgr_img_avg


In [18]:
def color_convert(cheek) :
    img=cheek
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

    sum = 0
    R = []
    G = []
    B = []
    for i in img :
        for j in i :
            R.append(j[0])
            G.append(j[1])
            B.append(j[2])

    R_sum = 0
    G_sum = 0
    B_sum = 0

    # 각 R, G, B의 합계 구하기
    for i in range(len(R)) :
        R_sum += R[i]
        G_sum += G[i]
        B_sum += B[i]

    R_avg = int(round((R_sum / len(R)), 0))  # R값 평균
    G_avg = int(round((G_sum / len(G)), 0))  # G값 평균
    B_avg = int(round((B_sum / len(B)), 0))  # B값 평균
    RGB_color = [R_avg, G_avg, B_avg]

    arr_RGB_color = np.array(RGB_color)
    float_arr_RGB_color = arr_RGB_color / 255
    float_tp_RGB_color = tuple(float_arr_RGB_color)
    HSV_color = matplotlib.colors.rgb_to_hsv(float_tp_RGB_color)
    HSV_color2 = np.array([round(HSV_color[0]*359, 3), round(HSV_color[1] * 100, 3)-4, round(HSV_color[2] * 100, 3)+8])
    HSV_color2 = list(HSV_color2)
    HSV_color2[0] = round(HSV_color2[0], 2)
    HSV_color2[1] = round(HSV_color2[1], 2)
    HSV_color2[2] = round(HSV_color2[2], 2)
    return HSV_color2


In [14]:
def save_img(file_name, skin_type) :

    if skin_type == 0 :
        print("spring")
    elif skin_type == 1:
        print("autumn")
    elif skin_type == 2:
        print("summer")
    elif skin_type == 3:
        print("winter")
    elif skin_type == -1:
        print("")
        print(file_name, "에러")

img = cv.imread("/content/drive/MyDrive/사계절_연예인 이미지 데이터셋/train data/F0001_IND_D_18_0_01.JPG") # 퍼스널컬러 알고싶은 사진
file_name="personal"
cheekimg = cheek(img)
cv.imwrite("cheek.JPG", cheekimg)
bar = skin_detector(cheekimg, file_name)
cv.imwrite("bar.JPG", bar)
bgr = color_convert2(bar)
hsv = color_convert(bar)

color_class = Color()
skin_type = color_class.color_classifier(hsv)
cv.imwrite("skin.JPG", bgr)
save_img(file_name, skin_type)

13.68 14.72
winter


In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt

# 피부 이미지 불러오기
cheek_image = cv.imread("cheek.JPG")

# 이미지를 RGB로 변환 (OpenCV는 BGR을 사용)
cheek_image_rgb = cv.cvtColor(cheek_image, cv.COLOR_BGR2RGB)

# 이미지 시각화
plt.imshow(cheek_image_rgb)
plt.axis("off")  # 축 제거
plt.title("Extracted Skin Image")
plt.show()


1. 일반인 이미지

In [ ]:
import os
import cv2 as cv

def save_img(file_name, skin_type) :

    if skin_type == 0 :
        print("spring")
    elif skin_type == 1:
        print("autumn")
    elif skin_type == 2:
        print("summer")
    elif skin_type == 3:
        print("winter")
    elif skin_type == -1:
        print("")
        print(file_name, "에러")

# 폴더 경로 지정
folder_path = "/content/drive/MyDrive/사계절_연예인 이미지 데이터셋/train data/"

# 폴더 내의 모든 파일을 가져오기 위해 glob 사용
import glob
file_list = glob.glob(folder_path + "*.JPG")

for file_path in file_list:
    # 파일명 추출 (확장자 제외)
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # 이미지 불러오기
    img = cv.imread(file_path)
    file_name="personal"
    cheekimg = cheek(img)
    cv.imwrite("cheek.JPG", cheekimg)
    bar = skin_detector(cheekimg, file_name)
    cv.imwrite("bar.JPG", bar)
    bgr = color_convert2(bar)
    hsv = color_convert(bar)

    color_class = Color()
    skin_type = color_class.color_classifier(hsv)
    cv.imwrite("skin.JPG", bgr)

      # save_img 함수 호출
    save_img(file_name, skin_type)

21.58 32.78
autumn
17.86 42.34
winter
23.45 54.59
autumn
personal 에러
0.0 12.0
winter
17.6 61.97
winter
18.05 48.17
winter
7.98 59.17
winter
20.98 48.98
autumn
19.04 50.59
winter
14.48 45.79
winter
16.51 34.09
winter
12.6 48.63
winter
28.34 35.91
autumn
23.64 22.69
autumn
19.38 49.14
winter
21.37 40.96
autumn
22.25 37.34
autumn
25.64 19.74
autumn
25.0 46.56
autumn
17.95 55.43
winter
19.36 42.38
winter
24.79 30.33
autumn
25.64 37.9
autumn
24.48 26.13
autumn
17.1 58.38
winter
19.04 50.04
winter
20.54 38.63
autumn
23.32 28.17
autumn
23.73 39.12
autumn
15.65 47.66
winter
15.44 39.75
winter
19.94 37.34
winter
19.38 44.36
winter
21.26 25.86
autumn
16.24 50.83
winter
15.84 46.6
winter
20.2 35.99
autumn
20.98 37.91
autumn
17.68 33.7
winter
15.39 48.49
winter
14.53 51.42
winter
17.45 44.42
winter
20.24 42.38
autumn
21.16 41.48
autumn
18.13 45.92
winter
20.81 47.84
autumn
9.74 53.91
winter
21.5 40.41
autumn
20.85 36.71
autumn
15.84 55.87
winter
18.6 63.55
winter
10.88 55.37
winter
25.31 37.34
aut

2. 연예인이미지(원본)

In [8]:
import os
import cv2 as cv

def save_img(file_name, skin_type) :

    if skin_type == 0 :
        print("spring")
    elif skin_type == 1:
        print("autumn")
    elif skin_type == 2:
        print("summer")
    elif skin_type == 3:
        print("winter")
    elif skin_type == -1:
        print("")
        print(file_name, "에러")

# 폴더 경로 지정
folder_path = "/content/drive/MyDrive/퍼스널컬러_jpg/웜톤_jpg"

# 폴더 내의 모든 파일을 가져오기 위해 glob 사용
import glob
file_list = glob.glob(folder_path + "*.JPG")

for file_path in file_list:
    # 파일명 추출 (확장자 제외)
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # 이미지 불러오기
    img = cv.imread(file_path)
    file_name="warm_personal"
    cheekimg = cheek(img)
    cv.imwrite("warm_cheek.JPG", cheekimg)
    bar = skin_detector(cheekimg, file_name)
    cv.imwrite("warm_bar.JPG", bar)
    bgr = color_convert2(bar)
    hsv = color_convert(bar)

    color_class = Color()
    skin_type = color_class.color_classifier(hsv)
    cv.imwrite("warm_skin.JPG", bgr)

      # save_img 함수 호출
    save_img(file_name, skin_type)

3. 연예인 이미지(cropped)

In [ ]:
import os
import cv2 as cv

def save_img(file_name, skin_type) :

    if skin_type == 0 :
        print("spring")
    elif skin_type == 1:
        print("autumn")
    elif skin_type == 2:
        print("summer")
    elif skin_type == 3:
        print("winter")
    elif skin_type == -1:
        print("")
        print(file_name, "에러")

# 폴더 경로 지정
folder_path = "/content/drive/MyDrive/cropped_face/웜톤"

# 폴더 내의 모든 파일을 가져오기 위해 glob 사용
import glob
file_list = glob.glob(folder_path + "*.jpg")

for file_path in file_list:
    # 파일명 추출 (확장자 제외)
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # 이미지 불러오기
    img = cv.imread(file_path)
    file_name="personal"
    cheekimg = cheek(img)
    cv.imwrite("cheek.jpg", cheekimg)
    bar = skin_detector(cheekimg, file_name)
    cv.imwrite("bar.jpg", bar)
    bgr = color_convert2(bar)
    hsv = color_convert(bar)

    color_class = Color()
    skin_type = color_class.color_classifier(hsv)
    cv.imwrite("skin.jpg", bgr)

      # save_img 함수 호출
    save_img(file_name, skin_type)

뺨

In [ ]:
import cv2 as cv
import dlib
import numpy as np

def visualize_cheek(image_path):
    # Load the image
    img = cv.imread(image_path)

    # Apply cheek() function to extract the cheek region
    cheekimg = cheek(img)

    # Display the original image and the cheek region side by side
    combined_image = np.hstack((img, cheekimg))
    cv.imshow("Original vs. Cheek", combined_image)
    cv.waitKey(0)
    cv.destroyAllWindows()

# Usage example
image_path = "path/to/your/image.jpg"  # Replace this with the actual path to your image
visualize_cheek(image_path)


계절 갯수

In [7]:
import os
import cv2 as cv

def save_img(file_name, skin_type):
    # 봄, 여름, 가을, 겨울 카운트를 위한 변수 초기화
    spring_count = 0
    summer_count = 0
    autumn_count = 0
    winter_count = 0

    if skin_type == 0:
        print("spring")
        spring_count += 1
    elif skin_type == 1:
        print("autumn")
        autumn_count += 1
    elif skin_type == 2:
        print("summer")
        summer_count += 1
    elif skin_type == 3:
        print("winter")
        winter_count += 1
    elif skin_type == -1:
        print("")
        print(file_name, "에러")

    # 봄, 여름, 가을, 겨울 이미지 각각의 개수 출력
    print("봄 이미지 개수:", spring_count)
    print("여름 이미지 개수:", summer_count)
    print("가을 이미지 개수:", autumn_count)
    print("겨울 이미지 개수:", winter_count)


NameError: ignored

파일 저장?

In [ ]:
import os
import cv2 as cv

def save_img(file_name, skin_type):
    if skin_type == 0:
        print("spring")
    elif skin_type == 1:
        print("autumn")
    elif skin_type == 2:
        print("summer")
    elif skin_type == 3:
        print("winter")
    elif skin_type == -1:
        print("")
        print(file_name, "에러")

# 폴더 경로 지정
folder_path = "/content/drive/MyDrive/사계절_연예인 이미지 데이터셋/train data/"

# 폴더 내의 모든 파일을 가져오기 위해 glob 사용
import glob
file_list = glob.glob(folder_path + "*.JPG")

for file_path in file_list:
    # 파일명 추출 (확장자 제외)
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # 이미지 불러오기
    img = cv.imread(file_path)
    cheekimg = cheek(img)
    cv.imwrite(file_name + "_cheek.JPG", cheekimg)
    bar = skin_detector(cheekimg, file_name)
    cv.imwrite(file_name + "_bar.JPG", bar)
    bgr = color_convert2(bar)
    hsv = color_convert(bar)

    color_class = Color()
    skin_type = color_class.color_classifier(hsv)
    cv.imwrite(file_name + "_skin.JPG", bgr)

    # save_img 함수 호출
    save_img(file_name, skin_type)
